# Loading the data

In [1]:
import sys
import os

# Get the parent directory path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the parent directory to the system path
sys.path.append(parent_dir)

In [3]:
import pandas as pd
import numpy as np
import wfdb
import torch

In [4]:
path = "../../data/ptb-xl/"

In [5]:
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')["filename_lr"]

In [51]:
signal, meta = wfdb.rdsamp(path+Y.iloc[0])

In [62]:
def mu_law_encode(audio, quantization_channels = 256):
    mu = float(quantization_channels - 1)
    quantize_space = np.linspace(-1, 1, quantization_channels)

    quantized = np.sign(audio) * np.log(1 + mu * np.abs(audio)) / np.log(mu + 1)
    quantized = np.digitize(quantized, quantize_space) - 1
    quantized = np.where(arr == -1, 0, quantized)

    return quantized

In [80]:
def get_data(filenames, padding = 0, channels = 256, batch_size = 1, start = 0):
    def load_audio(filename):
        signal, meta = wfdb.rdsamp(filename)
        return signal
    
    def one_hot_encode(data, channels=channels):
        one_hot = np.zeros((data.size, channels), dtype=float)
        one_hot[np.arange(data.size), data.ravel()] = 1

        return one_hot
    
    def mu_law_encode(audio, quantization_channels=channels):
        mu = float(quantization_channels - 1)
        quantize_space = np.linspace(-1, 1, quantization_channels)

        quantized = np.sign(audio) * np.log(1 + mu * np.abs(audio)) / np.log(mu + 1)
        quantized = np.digitize(quantized, quantize_space) - 1
        quantized = np.where(quantized == -1, 0, quantized)
        
        return quantized
    
    def _variable(data):
        tensor = torch.from_numpy(data).float()

        if torch.cuda.is_available():
            return torch.autograd.Variable(tensor.cuda())
        else:
            return torch.autograd.Variable(tensor)
    
    
    input_batch = []
    target_batch = []
    if start > len(filenames):
        print("9awditiha")
        return
    
    i = start
    
    while i < start + batch_size and i < len(Y):
        raw_audio = load_audio(path + filenames.iloc[i])

        input_ = raw_audio[:,0]
        input_ = np.pad(input_, [[padding, 0]], 'constant')
        input_ = mu_law_encode(input_)
        input_ = one_hot_encode(input_)

        target = raw_audio[:,1]
        target = mu_law_encode(target)
        
        input_batch.append(input_)
        target_batch.append(target)
        
        i+=1
    
    
    
    return _variable(np.array(input_batch)), _variable(np.array(target_batch))

In [81]:
def calc_receptive_fields(layer_size, stack_size):
        layers = [2 ** i for i in range(0, layer_size)] * stack_size
        num_receptive_fields = np.sum(layers)
        
        return int(num_receptive_fields)
    
def calc_output_size(x, receptive_fields):
    output_size = int(x.size(1)) - receptive_fields
    
    return output_size

In [82]:
import wavenet.config as config
from wavenet.model import WaveNet

In [83]:
layer_size = 5
stack_size = 1
batch_size = 30

In [84]:
receptive_fields = calc_receptive_fields(layer_size, stack_size);receptive_fields

31

In [85]:
wavenet = WaveNet(layer_size = layer_size, stack_size = stack_size,
                               in_channels = 256, res_channels = 512, lr = 0.01)

In [86]:
input_, target = get_data(Y, padding = receptive_fields, batch_size = batch_size)

In [88]:
target.long().view(-1).shape

torch.Size([30000])

In [89]:
output_size = calc_output_size(input_, receptive_fields);output_size

1000

In [40]:
out.view(-1, 256).shape

torch.Size([30000, 256])

In [45]:
target.shape

torch.Size([30, 1000])

In [68]:
target

tensor([[ 65.,  66.,  69.,  ..., 127., 143.,  77.],
        [210., 206., 201.,  ..., 233., 190.,  64.],
        [ 57.,  59.,  64.,  ...,  33.,  30.,  28.],
        ...,
        [ 59.,  57.,  51.,  ...,  59.,  62.,  59.],
        [ 44.,  44.,  45.,  ..., 204., 185.,  38.],
        [225., 224., 223.,  ...,   9.,   9.,   9.]])

In [69]:
for i in target.view(-1):
    if i <= 0:
        print(i)

tensor(0.)
tensor(-1.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(-1.)
tensor(0.)
tensor(-1.)
tensor(-1.)
tensor(-1.)
tensor(0.)
tensor(0.)
tensor(-1.)
tensor(-1.)


In [27]:
num_epoch = 2
for epoch in range(num_epoch):
    for i in range(0,len(Y),batch_size):
        print(i)
        input_, target = get_data(Y, padding = receptive_fields, batch_size =batch_size, start = i)
        print(input_.shape)
        print(target.shape)
        loss = wavenet.train(input_, target)
        
        print('[{0}/{1}] loss: {2}'.format(epoch + 1, num_epoch, loss))

0
torch.Size([30, 1031, 256])
torch.Size([30, 1000])


IndexError: Target -1 is out of bounds.

In [34]:
out = wavenet.generate(input_)

In [35]:
out.shape

torch.Size([30, 1000, 256])

In [21]:
out[0][0]

tensor([0.0041, 0.0038, 0.0040, 0.0038, 0.0038, 0.0036, 0.0038, 0.0037, 0.0039,
        0.0040, 0.0037, 0.0037, 0.0040, 0.0041, 0.0040, 0.0041, 0.0040, 0.0040,
        0.0038, 0.0037, 0.0039, 0.0037, 0.0037, 0.0038, 0.0037, 0.0039, 0.0038,
        0.0038, 0.0042, 0.0038, 0.0037, 0.0038, 0.0039, 0.0040, 0.0038, 0.0041,
        0.0042, 0.0038, 0.0041, 0.0038, 0.0041, 0.0037, 0.0038, 0.0041, 0.0037,
        0.0040, 0.0038, 0.0039, 0.0038, 0.0038, 0.0041, 0.0039, 0.0039, 0.0038,
        0.0038, 0.0040, 0.0042, 0.0041, 0.0040, 0.0038, 0.0037, 0.0036, 0.0039,
        0.0038, 0.0041, 0.0043, 0.0041, 0.0041, 0.0037, 0.0040, 0.0042, 0.0038,
        0.0037, 0.0040, 0.0039, 0.0039, 0.0037, 0.0038, 0.0040, 0.0040, 0.0037,
        0.0038, 0.0036, 0.0040, 0.0038, 0.0038, 0.0042, 0.0042, 0.0037, 0.0040,
        0.0037, 0.0040, 0.0039, 0.0041, 0.0038, 0.0040, 0.0040, 0.0040, 0.0040,
        0.0038, 0.0039, 0.0040, 0.0037, 0.0039, 0.0040, 0.0042, 0.0041, 0.0038,
        0.0040, 0.0038, 0.0041, 0.0041, 